In [1]:
!pip install pandas
!pip install sklearn


You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import logging
import glob
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 500)
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

# Download Data

In [3]:
# Ensure that the github-issues-data volume is mounted in /mnt
!ls -la .

total 140
drwxr-xr-x.  3 jovyan users    18 Jul 18 17:27 '~'
drwxr-xr-x.  6 jovyan users  4096 Jul 18 17:57  .
drwxr-xr-x. 11 jovyan users  4096 Jul 18 17:15  ..
-rw-r--r--.  1 jovyan users     0 Jul 18 17:57  body_pp.dpkl
-rw-r--r--.  1 jovyan users   223 Jul 18 17:15  Dockerfile
-rw-r--r--.  1 jovyan users   748 Jul 18 17:15  Dockerfile.estimator
-rw-r--r--.  1 jovyan users    77 Jul 18 17:15  environment_seldon_rest
lrwxrwxrwx.  1 jovyan users     7 Jul 18 17:48  github-issues-data -> jd_data
drwxr-xr-x.  2 jovyan users    39 Jul 18 17:19  .ipynb_checkpoints
-rw-r--r--.  1 jovyan users  1243 Jul 18 17:15  IssueSummarization.py
drwxr-xr-x.  2 jovyan users    56 Jul 18 17:47  jd_data
-rw-r--r--.  1 jovyan users  4285 Jul 18 17:15  Makefile
-rw-r--r--.  1 jovyan users    74 Jul 18 17:15  requirements.txt
-rw-r--r--.  1 jovyan users 14833 Jul 18 17:15  seq2seq_utils.py
drwxr-xr-x.  3 jovyan users    32 Jul 18 17:42  test_data
-rw-r--r--.  1 jovyan users 10766 Jul 18 17:15  trainer.py
-r

In [4]:
# Set path for data dir
%env DATA_DIR=jd_data
!whoami
!pwd
!rm -rf ${DATA_DIR}
!mkdir ${DATA_DIR}
!ls -ltr
!cd ${DATA_DIR}
!ls -ltr
!pwd
!ls -ltr ${DATA_DIR}
!echo DONE

env: DATA_DIR=jd_data
jovyan
/home/jovyan/examples/github_issue_summarization/notebooks
total 132
-rw-r--r--. 1 jovyan users   748 Jul 18 17:15  Dockerfile.estimator
-rw-r--r--. 1 jovyan users   223 Jul 18 17:15  Dockerfile
-rw-r--r--. 1 jovyan users 14833 Jul 18 17:15  seq2seq_utils.py
-rw-r--r--. 1 jovyan users    74 Jul 18 17:15  requirements.txt
-rw-r--r--. 1 jovyan users  4285 Jul 18 17:15  Makefile
-rw-r--r--. 1 jovyan users  1243 Jul 18 17:15  IssueSummarization.py
-rw-r--r--. 1 jovyan users    77 Jul 18 17:15  environment_seldon_rest
-rw-r--r--. 1 jovyan users  2546 Jul 18 17:15  train_test.py
-rw-r--r--. 1 jovyan users  6548 Jul 18 17:15  train.py
-rw-r--r--. 1 jovyan users 10766 Jul 18 17:15  trainer.py
drwxr-xr-x. 3 jovyan users    18 Jul 18 17:27 '~'
drwxr-xr-x. 3 jovyan users    32 Jul 18 17:42  test_data
lrwxrwxrwx. 1 jovyan users     7 Jul 18 17:48  github-issues-data -> jd_data
-rw-r--r--. 1 jovyan users 61918 Jul 18 17:57  Training.ipynb
-rw-r--r--. 1 jovyan users     

In [5]:
# Download the github-issues.zip training data to /mnt/github-issues-data
!mkdir -p ${DATA_DIR}
!wget --directory-prefix=${DATA_DIR} https://storage.googleapis.com/kubeflow-examples/github-issue-summarization-data/github-issues.zip
!ls -ltr
    
# Unzip the file into /mnt/github-issues-data directory
!unzip ${DATA_DIR}/github-issues.zip -d ${DATA_DIR}

--2019-07-18 17:59:04--  https://storage.googleapis.com/kubeflow-examples/github-issue-summarization-data/github-issues.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.210.240, 2a00:1450:4009:817::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.210.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027424178 (980M) [application/zip]
Saving to: ‘jd_data/github-issues.zip’

github-issues.zip   100%[===================>] 979.83M  24.7MB/s    in 41s     

2019-07-18 17:59:45 (24.2 MB/s) - ‘jd_data/github-issues.zip’ saved [1027424178/1027424178]

total 92
-rw-r--r--. 1 jovyan users   748 Jul 18 17:15  Dockerfile.estimator
-rw-r--r--. 1 jovyan users   223 Jul 18 17:15  Dockerfile
-rw-r--r--. 1 jovyan users 14833 Jul 18 17:15  seq2seq_utils.py
-rw-r--r--. 1 jovyan users    74 Jul 18 17:15  requirements.txt
-rw-r--r--. 1 jovyan users  4285 Jul 18 17:15  Makefile
-rw-r--r--. 1 jovyan users  1243 Jul 18 17:15  IssueSu

In [6]:
# Create a symlink from <current_directory>/github-issues-data to /mnt/github-issues-data
!rm -rf github-issues-data
!ln -sf ${DATA_DIR} github-issues-data
!ls -ltr

total 92
-rw-r--r--. 1 jovyan users   748 Jul 18 17:15  Dockerfile.estimator
-rw-r--r--. 1 jovyan users   223 Jul 18 17:15  Dockerfile
-rw-r--r--. 1 jovyan users 14833 Jul 18 17:15  seq2seq_utils.py
-rw-r--r--. 1 jovyan users    74 Jul 18 17:15  requirements.txt
-rw-r--r--. 1 jovyan users  4285 Jul 18 17:15  Makefile
-rw-r--r--. 1 jovyan users  1243 Jul 18 17:15  IssueSummarization.py
-rw-r--r--. 1 jovyan users    77 Jul 18 17:15  environment_seldon_rest
-rw-r--r--. 1 jovyan users  2546 Jul 18 17:15  train_test.py
-rw-r--r--. 1 jovyan users  6548 Jul 18 17:15  train.py
-rw-r--r--. 1 jovyan users 10766 Jul 18 17:15  trainer.py
drwxr-xr-x. 3 jovyan users    18 Jul 18 17:27 '~'
drwxr-xr-x. 3 jovyan users    32 Jul 18 17:42  test_data
-rw-r--r--. 1 jovyan users     0 Jul 18 17:57  body_pp.dpkl
-rw-r--r--. 1 jovyan users 21891 Jul 18 17:59  Training.ipynb
drwxr-xr-x. 2 jovyan users    56 Jul 18 17:59  jd_data
lrwxrwxrwx. 1 jovyan users     7 Jul 18 18:00  github-issues-data -> jd_data


In [7]:
# Make sure that the github-issues-data symlink is created
!ls -lh github-issues-data/github_issues.csv

-rw-r--r--. 1 jovyan users 2.7G Jan 17  2018 github-issues-data/github_issues.csv


# Process Data

Split data into train and test set and preview data

In [8]:
data_file='github-issues-data/github_issues.csv'


# read in data sample 2000 rows (for speed of tutorial)
# Set this to False to train on the entire dataset
use_sample_data=True

if use_sample_data:
    training_data_size=2000
    traindf, testdf = train_test_split(pd.read_csv(data_file).sample(n=training_data_size), 
                                   test_size=.10)
else:
    traindf, testdf = train_test_split(pd.read_csv(data_file),test_size=.10)


#print out stats about shape of data
print(f'Train: {traindf.shape[0]:,} rows {traindf.shape[1]:,} columns')
print(f'Test: {testdf.shape[0]:,} rows {testdf.shape[1]:,} columns')

# preview data
traindf.head(3)

Train: 1,800 rows 3 columns
Test: 200 rows 3 columns


,issue_url,issue_title,body
4808004,"""https://github.com/turretcss/turretcss/issues/14""",typo it's > its,makes element cover it's container using .cover; mixin it's > its
1151120,"""https://github.com/Brewskey/brewskey.js-api/issues/25""",flow lib export is broken,"when we use the lib in other project. import types { sometipe } works correct. but daoapi.createfilter -or whatever from daoapi is not typechecked. need to figure out correct way to export default in flow module declarations. there are some work around here: https://github.com/facebook/flow/issues/1806 i was trying from that: declare type daoapi = { createfilter: ... ..., } declare export default daoapi; but seems this doesn't work :/"
4904563,"""https://github.com/bluekyu/panda3d/issues/1""",add additional thirdparties,- directcam - vision - tiff - openexr see also https://github.com/bluekyu/panda3d-thirdparty/issues/2


**Convert to lists in preparation for modeling**

In [9]:
train_body_raw = traindf.body.tolist()
train_title_raw = traindf.issue_title.tolist()
#preview output of first element
train_body_raw[0]

"makes element cover it's container using .cover; mixin it's > its"

# Pre-Process Data For Deep Learning

See [this repo](https://github.com/hamelsmu/ktext) for documentation on the ktext package

In [10]:
!pip install ktext
%reload_ext autoreload
%autoreload 2
from ktext.preprocess import processor

You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [11]:
%%time
# Clean, tokenize, and apply padding / truncating such that each document length = 70
#  also, retain only the top 8,000 words in the vocabulary and set the remaining words
#  to 1 which will become common index for rare words 
body_pp = processor(keep_n=8000, padding_maxlen=70)
train_body_vecs = body_pp.fit_transform(train_body_raw)
print('\noriginal string:\n', train_body_raw[0], '\n')
print('after pre-processing:\n', train_body_vecs[0], '\n')


original string:
 makes element cover it's container using .cover; mixin it's > its 

after pre-processing:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  868  629 2739   10   30  387   45 2739 2342   10   30  304] 

CPU times: user 156 ms, sys: 333 ms, total: 489 ms
Wall time: 914 ms


#### Look at one example of processed issue bodies

In [12]:
print('\noriginal string:\n', train_body_raw[0], '\n')
print('after pre-processing:\n', train_body_vecs[0], '\n')


original string:
 makes element cover it's container using .cover; mixin it's > its 



NameError: name 'train_body_vecs' is not defined

In [13]:
# Instantiate a text processor for the titles, with some different parameters
#  append_indicators = True appends the tokens '_start_' and '_end_' to each
#                      document
#  padding = 'post' means that zero padding is appended to the end of the 
#             of the document (as opposed to the default which is 'pre')
title_pp = processor(append_indicators=True, keep_n=4500, 
                     padding_maxlen=12, padding ='post')

# process the title data
train_title_vecs = title_pp.fit_transform(train_title_raw)

#### Look at one example of processed issue titles

In [14]:
print('\noriginal string:\n', train_title_raw[0])
print('after pre-processing:\n', train_title_vecs[0])


original string:
 typo it's > its
after pre-processing:
 [  2 292  43  59 446   3   0   0   0   0   0   0]


Serialize all of this to disk for later use

In [15]:
import dill as dpickle
import numpy as np

# Save the preprocessor
with open('body_pp.dpkl', 'wb') as f:
    dpickle.dump(body_pp, f)

with open('title_pp.dpkl', 'wb') as f:
    dpickle.dump(title_pp, f)

# Save the processed data
np.save('train_title_vecs.npy', train_title_vecs)
np.save('train_body_vecs.npy', train_body_vecs)

NameError: name 'body_pp' is not defined

# Define Model Architecture

### Load the data from disk into variables

In [ ]:
from seq2seq_utils import load_decoder_inputs, load_encoder_inputs, load_text_processor

In [ ]:
encoder_input_data, doc_length = load_encoder_inputs('train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs('train_title_vecs.npy')

In [ ]:
num_encoder_tokens, body_pp = load_text_processor('body_pp.dpkl')
num_decoder_tokens, title_pp = load_text_processor('title_pp.dpkl')

### Define Model Architecture

In [ ]:
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from keras import optimizers

In [ ]:
#arbitrarly set latent dimension for embedding and hidden units
latent_dim = 300

##### Define Model Architecture ######

########################
#### Encoder Model ####
encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

# Word embeding for encoder (ex: Issue Body)
x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word-Embedding', mask_zero=False)(encoder_inputs)
x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

# Intermediate GRU layer (optional)
#x = GRU(latent_dim, name='Encoder-Intermediate-GRU', return_sequences=True)(x)
#x = BatchNormalization(name='Encoder-Batchnorm-2')(x)

# We do not need the `encoder_output` just the hidden state.
_, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

# Encapsulate the encoder as a separate entity so we can just 
#  encode without decoding if we want to.
encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

########################
#### Decoder Model ####
decoder_inputs = Input(shape=(None,), name='Decoder-Input')  # for teacher forcing

# Word Embedding For Decoder (ex: Issue Titles)
dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

# Set up the decoder, using `decoder_state_input` as initial state.
decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

# Dense layer for prediction
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)

########################
#### Seq2Seq Model ####

#seq2seq_decoder_out = decoder_model([decoder_inputs, seq2seq_encoder_out])
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=0.001), loss='sparse_categorical_crossentropy')

** Examine Model Architecture Summary **

In [ ]:
from seq2seq_utils import viz_model_architecture
seq2seq_Model.summary()
viz_model_architecture(seq2seq_Model)

# Train Model

In [ ]:
from keras.callbacks import CSVLogger, ModelCheckpoint

script_name_base = 'tutorial_seq2seq'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                   save_best_only=True)

batch_size = 1200
epochs = 7
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12, callbacks=[csv_logger, model_checkpoint])

In [ ]:
#save model
seq2seq_Model.save('seq2seq_model_tutorial.h5')

# See Example Results On Holdout Set

It is useful to see examples of real predictions on a holdout set to get a sense of the performance of the model.  We will also evaluate the model numerically in a following section.

In [ ]:
from seq2seq_utils import Seq2Seq_Inference
seq2seq_inf = Seq2Seq_Inference(encoder_preprocessor=body_pp,
                                 decoder_preprocessor=title_pp,
                                 seq2seq_model=seq2seq_Model)

In [ ]:
# this method displays the predictions on random rows of the holdout set
seq2seq_inf.demo_model_predictions(n=50, issue_df=testdf)

# Evaluate Model: BLEU Score

For machine-translation tasks such as this one, it is common to measure the accuracy of results using the [BLEU Score](https://en.wikipedia.org/wiki/BLEU).  The convenience function illustrated below uses [NLTK's corpus_bleu](https://www.nltk.org/api/nltk.translate.html#nltk.translate.bleu_score.corpus_bleu).  The output of the below convenience function is an Average of BlEU-1, BLEU-2, BLEU-3 and BLEU-4. 

In [ ]:
#convenience function that generates predictions on holdout set and calculates BLEU Score

bleu_score = seq2seq_inf.evaluate_model(holdout_bodies=testdf.body.tolist(), 
                                        holdout_titles=testdf.issue_title.tolist(), 
                                        max_len_title=12)

In [ ]:
print(f'BLEU Score (avg of BLUE 1-4) on Holdout Set: {bleu_score * 100}')